In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.

import os
import urllib
from pyspark.sql import SparkSession

# URL of the CSV file
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Local file path where the data will be downloaded
local_directory = "/path/to/your/local/folder"
local_file_path = os.path.join(local_directory, "home_sales_revised.csv")

# Create the local directory if it does not exist
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# Download the file using urllib
urllib.request.urlretrieve(url, local_file_path)

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Read the data from the local file into a DataFrame
df = spark.read.csv("file://" + local_file_path, header=True, inferSchema=True)

# Show the DataFrame
df.show()

23/07/31 13:49:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


OSError: [Errno 30] Read-only file system: '/path'

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales_view")


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePrice").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for a four-bedroom house sold in each year
query = """
SELECT YEAR(date) as year, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePriceByYearBuilt").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for homes with 3 bedrooms and 3 bathrooms by year built
query = """
SELECT date_built as year_built, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()

In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

from pyspark.sql import SparkSession
from pyspark.sql.functions import round

# Create a SparkSession
spark = SparkSession.builder.appName("AveragePriceByYearBuilt").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# SQL query to calculate average price for homes with 3 bedrooms, 3 bathrooms, two floors,
# and are greater than or equal to 2,000 square feet, by year built
query = """
SELECT date_built as year_built, ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

# Execute the query and store the results in a new DataFrame
result_df = spark.sql(query)

# Show the results
result_df.show()

In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Measure the start time of the query execution
start_time = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000
query = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df = spark.sql(query)

# Extract the "average_price" value from the DataFrame result
view_rating = float(result_df.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time = time.time()

# Show the "view" rating for the average price (rounded to two decimal places)
print("View Rating for Average Price (rounded to two decimal places): {:.2f}".format(view_rating))

# Calculate and display the runtime of the query
runtime = end_time - start_time
print("Query Runtime:", runtime, "seconds")

In [ ]:
# 7. Cache the the temporary table home_sales.
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Measure the start time of the query execution
start_time = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000
query = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df = spark.sql(query)

# Extract the "average_price" value from the DataFrame result
view_rating = float(result_df.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time = time.time()

# Show the "view" rating for the average price (rounded to two decimal places)
print("View Rating for Average Price (rounded to two decimal places): {:.2f}".format(view_rating))

# Calculate and display the runtime of the query
runtime = end_time - start_time
print("Query Runtime:", runtime, "seconds")

In [ ]:
# 8. Check if the table is cached.
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Check if the table is cached
is_cached = spark.catalog.isCached("home_sales_view")

# Print the result
print("Is 'home_sales_view' cached?", is_cached)

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ViewRating").getOrCreate()

# Create a temporary view for the DataFrame
df.createOrReplaceTempView("home_sales_view")

# Cache the temporary table
spark.catalog.cacheTable("home_sales_view")

# Measure the start time of the query execution
start_time_cached = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000 using the cached data
query_cached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_cached = spark.sql(query_cached)

# Extract the "average_price" value from the DataFrame result
view_rating_cached = float(result_df_cached.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time_cached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) using cached data
print("View Rating for Average Price (rounded to two decimal places) with cached data: {:.2f}".format(view_rating_cached))

# Calculate and display the runtime of the query using cached data
runtime_cached = end_time_cached - start_time_cached
print("Query Runtime with cached data:", runtime_cached, "seconds")

# Measure the start time of the query execution without using cached data
start_time_uncached = time.time()

# SQL query to calculate average price for homes greater than or equal to $350,000 without using cached data
query_uncached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM home_sales_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_uncached = spark.sql(query_uncached)

# Extract the "average_price" value from the DataFrame result
view_rating_uncached = float(result_df_uncached.collect()[0]["average_price"])

# Measure the end time of the query execution without using cached data
end_time_uncached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) without using cached data
print("View Rating for Average Price (rounded to two decimal places) without cached data: {:.2f}".format(view_rating_uncached))

# Calculate and display the runtime of the query without using cached data
runtime_uncached = end_time_uncached - start_time_uncached
print("Query Runtime without cached data:", runtime_uncached, "seconds")


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Assuming you have the DataFrame 'df' created and formatted as Parquet from the previous code

# Write the DataFrame to Parquet format and partition by the "date_built" field
df.write.partitionBy("date_built").parquet("/path/to/parquet_output")


In [ ]:
# 11. Read the formatted parquet data.
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("ReadParquetData").getOrCreate()

# Read the Parquet data with partitions
parquet_data = spark.read.format("parquet").load("/path/to/parquet_output")

# Show the data
parquet_data.show()

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_data_view")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 
# Register the DataFrame as a temporary view
parquet_data.createOrReplaceTempView("parquet_data_view")

# Measure the start time of the query execution
start_time_uncached = time.time()

# SQL query to calculate the average price for homes greater than or equal to $350,000
query_uncached = """
SELECT ROUND(AVG(price), 2) as average_price
FROM parquet_data_view
WHERE price >= 350000
"""

# Execute the query and store the result in a new DataFrame
result_df_uncached = spark.sql(query_uncached)

# Extract the "average_price" value from the DataFrame result
view_rating_uncached = float(result_df_uncached.collect()[0]["average_price"])

# Measure the end time of the query execution
end_time_uncached = time.time()

# Show the "view" rating for the average price (rounded to two decimal places) without using cached data
print("View Rating for Average Price (rounded to two decimal places) without cached data: {:.2f}".format(view_rating_uncached))

# Calculate and display the runtime of the query without using cached data
runtime_uncached = end_time_uncached - start_time_uncached
print("Query Runtime without cached data:", runtime_uncached, "seconds")

In [ ]:
# 14. Uncache the home_sales temporary table.

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Unpersist the cached DataFrame
df.unpersist()

# Check if the table is still cached (should return False)
is_cached = df.is_cached
print("Is 'home_sales' cached?", is_cached)

In [ ]:
# 15. Check if the home_sales is no longer cached

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Assuming you have the DataFrame 'df' created and cached as the DataFrame "home_sales"

# Unpersist the cached DataFrame
df.unpersist()

# Check if the DataFrame is still cached
is_cached = df.is_cached

if not is_cached:
    print("DataFrame 'home_sales' is no longer cached.")
else:
    print("DataFrame 'home_sales' is still cached.")
